In [5]:
import pandas as pd
import numpy as np

# to reduce amount of repeating code from notebook to another there is a lib of
#often used functions called
import sys
sys.path.insert(0, "C:/workspace/data-science/perpetual/second_investigation_wave")
from data_transformers import *

### Reading amm positions with all required changes

In [6]:
amm_positions_df = pd.read_csv("amm_positions_pure.csv")
amm_positions_df = transform_18_decimals_to_float(amm_positions_df, ["margin", "position_size", "open_notional",
                                                                     "trading_volume", "leverage", "entry_price",
                                                                     "realized_pnl", "unrealized_pnl", "funding_payment",
                                                                     "fee", "bad_debt", "liquidation_penalty",
                                                                     "total_pnl_amount"])
transform_amm_address_to_name(amm_positions_df)
amm_positions_df["time"] = pd.to_datetime(amm_positions_df["timestamp"], unit='s')

### Reading general AMM info

In [7]:
amm_pure_df = pd.read_csv("amm_pure_df.csv")
amm_pure_df = transform_18_decimals_to_float(amm_pure_df, ["position_balance", "open_interest_size",
                                                            "open_interest_notional", "trading_volume", 
                                                            "quote_asset_reserve", "base_asset_reserve"])
amm_pure_df["time"] = pd.to_datetime(amm_pure_df["timestamp"], unit='s')

### Reading funding rate updates

In [8]:
funding_rate_updates_df = pd.read_csv("funding_rate_updated_events_pure.csv")
funding_rate_updates_df = transform_18_decimals_to_float(funding_rate_updates_df, ["rate", "underlying_price"])
transform_amm_address_to_name(funding_rate_updates_df)
funding_rate_updates_df["time"] = pd.to_datetime(funding_rate_updates_df["timestamp"], unit='s')

### Reading liquidations events

In [9]:
liquidation_events_df = pd.read_csv("liquidation_events_pure.csv")
liquidation_events_df = transform_18_decimals_to_float(liquidation_events_df, ["position_notional", "position_size",
                                                                               "liquidation_fee", "bad_debt"])
transform_amm_address_to_name(liquidation_events_df)
liquidation_events_df["time"] = pd.to_datetime(liquidation_events_df["timestamp"], unit='s')

### Reading margin changes

In [10]:
margin_changes_df = pd.read_csv("margin_changed_events_pure.csv")
margin_changes_df = transform_18_decimals_to_float(margin_changes_df, ["amount", "funding_payment"])
transform_amm_address_to_name(margin_changes_df)
margin_changes_df["time"] = pd.to_datetime(margin_changes_df["timestamp"], unit='s')

### Reading position changes

In [11]:
position_changes_df = pd.read_csv("position_changed_events_pure.csv")
position_changes_df = transform_18_decimals_to_float(position_changes_df, ["margin", "position_notional", 
                                                                           "exchanged_position_size", "fee", 
                                                                           "position_size_after", "realized_pnl",
                                                                           "unrealized_pnl_after", "bad_debt",
                                                                           "liquidation_penalty", "spot_price",
                                                                           "funding_payment"])
transform_amm_address_to_name(position_changes_df)
position_changes_df = transform_id_to_hash_and_change(position_changes_df)
position_changes_df["time"] = pd.to_datetime(position_changes_df["timestamp"], unit='s')

C:\Users\Zbook\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (6,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Setting intermediate opens/closes made by traders

In [13]:
general_changes_opens_closes_list = []

for pool in position_changes_df["amm_name"].unique():
    print("start of " + pool + " analysis")
    
    #   take only story of current
    pool_position_changes_df = position_changes_df[position_changes_df["amm_name"] == pool]
    pool_position_changes_df["open_id"] = np.zeros(len(pool_position_changes_df))
    pool_traders_count = len(pool_position_changes_df["trader"].unique())
    cur_iteration = 0
    
    #   set array of further combinable opens and closes into one big story
    pool_opens_closes_list = []
    
    for trader in pool_position_changes_df["trader"].unique():
        #   out amm story collect activity of the current trader
        trader_pool_position_changes_df = pool_position_changes_df[pool_position_changes_df["trader"] == trader]
        trader_pool_opens_df = trader_pool_position_changes_df[
            trader_pool_position_changes_df["exchanged_position_size"] == trader_pool_position_changes_df["position_size_after"]
        ]
        
        #   set column for opening transactions and then perform ffill to track when position will be closed
        trader_pool_position_changes_df.loc[trader_pool_opens_df.index, 'open_id'] = np.arange(1, len(trader_pool_opens_df) + 1)
        trader_pool_position_changes_df['open_id'].replace(to_replace=0, method='ffill', inplace=True)
        pool_opens_closes_list.append(trader_pool_position_changes_df)
        
        print('finished ' + str(cur_iteration) + ' iteration out of ' + str(pool_traders_count))
        cur_iteration += 1
    
    #   concatenate all opens and closes of the pool and then add pool opens and closes to further make
    # a global opens and closes story
    pool_opens_closes_df = pd.concat(pool_opens_closes_list)
    general_changes_opens_closes_list.append(pool_opens_closes_df)    

start of BTC/USDC analysis


<ipython-input-13-e8900b74e6b8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pool_position_changes_df["open_id"] = np.zeros(len(pool_position_changes_df))
C:\Users\Zbook\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


finished 0 iteration out of 5034
finished 1 iteration out of 5034
finished 2 iteration out of 5034
finished 3 iteration out of 5034
finished 4 iteration out of 5034
finished 5 iteration out of 5034
finished 6 iteration out of 5034
finished 7 iteration out of 5034
finished 8 iteration out of 5034
finished 9 iteration out of 5034
finished 10 iteration out of 5034
finished 11 iteration out of 5034
finished 12 iteration out of 5034
finished 13 iteration out of 5034
finished 14 iteration out of 5034
finished 15 iteration out of 5034
finished 16 iteration out of 5034
finished 17 iteration out of 5034
finished 18 iteration out of 5034
finished 19 iteration out of 5034
finished 20 iteration out of 5034
finished 21 iteration out of 5034
finished 22 iteration out of 5034
finished 23 iteration out of 5034
finished 24 iteration out of 5034
finished 25 iteration out of 5034
finished 26 iteration out of 5034
finished 27 iteration out of 5034
finished 28 iteration out of 5034
finished 29 iteration ou

In [14]:
general_changes_opens_closes_df = pd.concat(general_changes_opens_closes_list)
general_changes_opens_closes_df.to_csv("changes_opens_closes_df.csv")